In [ ]:
import pandas as pd
import numpy as np  

In [ ]:
#importing dataset
training_data = pd.read_csv("emnist-letters-train.csv",header=None)
test_data = pd.read_csv("emnist-letters-test (1).csv",header = None)
l1=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [ ]:
#creating 26 dataset for different letters
datasets = []

for i in range(26):
    datasets.append(training_data.copy())
for i in range(1,27):
    for j in range(88800):
        if datasets[i-1].iloc[j,0] == i:
            datasets[i-1].iloc[j,0]=1
        else:
            datasets[i-1].iloc[j,0]=0   

train_dict={}
for i in range(1,27):
    X = datasets[i-1].iloc[:,1:785]
    y = datasets[i-1].iloc[:,0] 
    y = np.array(y)
    y = y.reshape(y.shape[0],1)
    
    train_dict[l1[i-1]] = [X,y]



datasets1 = []

for i in range(26):
    datasets1.append(test_data.copy())
for i in range(1,27):
    print(f"Iteration{i}")
    for j in range(14799):
        if datasets1[i-1].iloc[j,0] == i:
            datasets1[i-1].iloc[j,0]=1
        else:
            datasets1[i-1].iloc[j,0]=0   

test_dict={}
for i in range(1,27):
    X = datasets1[i-1].iloc[:,1:785]
    y = datasets1[i-1].iloc[:,0]  
    y = np.array(y)
    y = y.reshape(y.shape[0],1)
    
    test_dict[l1[i-1]] = [X,y]


In [ ]:
#2 layered neural network delta and grad calculations
def grad(X,y,theta1,theta2,theta3,itr,alpha,lmd):
    y = np.array(y)
    y = y.reshape(y.shape[0],1)
    delta1=0
    delta2=0
    delta3=0
        
        
    m = y.shape[0]
    a1 = X
    for i in range(itr):
            
        z2 = X@(theta1)
        a2 = 1/(1+np.exp(z2*(-1)))
           
            
        z3 = a2@(theta2)
        a3 = 1/(1+np.exp(z3*(-1)))
         
        z4 = a3@(theta3)
        h = 1/(1+np.exp(z4*(-1)))
        
        del4 = h-y
        del3 = (del4@theta3.T)*(a3*(1-a3))
        
        a2 = 1/(1+np.exp(z2*(-1)))
        del2 = ((del3@(theta2.T))*(a2*(1-a2)))
           
            
        delta1 = delta1 + (del2.T)@(a1) 
        delta2 = delta2 + (del3.T)@(a2) 
        delta3 = delta3 + (del4.T)@(a3) 
            
        d1 = ((1/m)*(delta1.T)) + (lmd/m)*theta1
        d2 = ((1/m)*(delta2.T)) + (lmd/m)*theta2
        d3 = ((1/m)*(delta3.T)) + (lmd/m)*theta3
            
        d1[:,0] = (1/m)*(delta1.T[:,0])
        d2[:,0] = (1/m)*(delta2.T[:,0])
        d3[:,0] = (1/m)*(delta3.T[:,0])
            
        theta1 = theta1 - (alpha)*(d1)
        theta2 = theta2 - (alpha)*(d2)
        theta3 = theta3 - (alpha)*(d3)
            

            
            
    return theta1,theta2,theta3,h

In [ ]:
#testing and checking accuracy
theta_history = {}
h_history ={}

for i in range(1,27):
    alphabet = l1[i-1]
    X_train = train_dict[l1[i-1]][0]
    X_train = np.c_[np.ones((X_train.shape[0],1)), X_train]
    y_train = train_dict[l1[i-1]][1]
    theta1 = np.ones((785,10))
    theta2 = np.ones((10,10))
    theta3 = np.ones((10,1))
    theta1,theta2,theta3,h =grad(X_train,y_train,theta1,theta2,theta3,50,0.1,0.1)
    theta_history[alphabet] = [theta1,theta2,theta3]
    h_history[alphabet] = [h]
    
    c = 0
    y_train = np.array(y_train)
    y_train = y_train.reshape(88800,1)
    if np.logical_and(h.sum()>=0.5 , y[j] ==1):
            c += 1
    if np.logical_and(h.sum()<0.5 , y[j]==0):
            c += 1
    print("Accuracy for alphabet " + alphabet + str( (c/88800)*100))

In [ ]:
#testing accuracy on test data
for i in range(1,27):
    alphabet = l1[i-1]
    X_test = test_dict[l1[i-1]][0]
    X_test = np.c_[np.ones((X_test.shape[0],1)), X_test]
    y_test = test_dict[l1[i-1]][1]
    theta1 = theta_history[alphabet][0]
    theta2 = theta_history[alphabet][1]
    theta3 = theta_history[alphabet][2] 
    z2 = X_test@(theta1)
    a2 = 1/(1+np.exp(z2*(-1)))
    z3 = a2@(theta2)
    a3 = 1/(1+np.exp(z3*(-1)))
    z4 = a3@(theta3)
    h = 1/(1+np.exp(z4*(-1)))

    c = 0
    y_test = np.array(y_test)
    y_test = y_test.reshape(14800,1)
    if np.logical_and(h.sum()>=0.5 , y[j] ==1):
            c += 1
    if np.logical_and(h.sum()<0.5 , y[j]==0):
            c += 1
    print("Accuracy for alphabet " + alphabet+ str( (c/X_test.shape[0])*100))